In [34]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, Tool
from agents.mcp import MCPServerStdio
from agents import Agent, Runner, OpenAIChatCompletionsModel
from openai import AsyncOpenAI
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account
import yfinance as yf
import asyncio
from IPython.display import Markdown, display
import sys
import nest_asyncio

load_dotenv(override=True)

True

In [77]:
client = AsyncOpenAI(
    api_key=os.getenv("GEMINI_ACCESS_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
model = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash",
    openai_client=client
)

In [62]:
market_mcp={"command":"uv","args":["run","market_server.py"]}

trader_mcp_server_params=[
    {"command":"uv","args":["run","accounts_server.py"]},
    {"command":"uv","args":["run","push_server.py"]},
    market_mcp
]

reasearcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {
        "command": "npx",
        "args": ["-y", "@mcptools/mcp-tavily"],
        "env": {"TAVILY_API_KEY": os.getenv("TAVILY_SEARCH_API_KEY")}
    }
]

In [63]:
## Now create a MCPServerStdio for both
researcher_mcp_servers=[MCPServerStdio(params,client_session_timeout_seconds=30) for params in reasearcher_mcp_server_params]
trader_mcp_servers=[MCPServerStdio(params,client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = researcher_mcp_servers + trader_mcp_servers

In [78]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
    look for possible trading opportunities, and help with research.
    Based on the request, you carry out necessary research and respond with your findings.
    Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
    If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
    The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    """
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model=model,
        mcp_servers=mcp_servers,
    )
    return researcher

In [65]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [52]:
# research_question = "What's the latest news on ITC?"

# for server in researcher_mcp_servers:
#     await server.connect()
# researcher = await get_researcher(researcher_mcp_servers)
# result = await Runner.run(researcher, research_question, max_turns=30)
# display(Markdown(result.final_output))

In [66]:
my_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Himanshu").reset(my_initial_strategy)

display(Markdown(await read_accounts_resource("Himanshu")))
display(Markdown(await read_strategy_resource("Himanshu")))

{"name": "himanshu", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-26 23:00:43", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

<h3>Let's create a trader agent</h3>

In [67]:
agent_name = "Himanshu"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

print(instructions)


You are a trader that manages a portfolio of shares. Your name is Himanshu and your account is under your name, Himanshu.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "himanshu", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-26 23:00:43", 10000.0], ["2025-08-26 23:00:49", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and think

<h3>Run the trader</h3>

In [79]:

# async with asyncio.TaskGroup() as tg:
#     for server in mcp_servers:
#         tg.create_task(server.connect())

# # Now proceed to create tools and agents
# researcher_tool = await get_researcher_tool(researcher_mcp_servers)
# trader = Agent(
#     name=agent_name,
#     instructions=instructions,
#     tools=[researcher_tool],
#     mcp_servers=trader_mcp_servers,
#     model=model,
# )


# result = await Runner.run(trader, prompt, max_turns=10)
# print(result.final_output)
# display(Markdown(result.final_output))


# Connect servers concurrently before agent/tools instantiation
async def connect_all_servers(mcp_servers):
    # Gather all server connections
    await asyncio.gather(*(server.connect() for server in mcp_servers))

import nest_asyncio
nest_asyncio.apply()



async def main():
    await connect_all_servers(mcp_servers)
    researcher_tool = await get_researcher_tool(researcher_mcp_servers)
    trader = Agent(
        name=agent_name,
        instructions=instructions,
        tools=[researcher_tool],
        mcp_servers=trader_mcp_servers,
        model=model,
    )
    result = await Runner.run(trader, prompt)
    print(result.final_output)
    sys.stdout.flush()
    display(Markdown(result.final_output))
    return result

# Run
result_main = await main()
# print("Result::",result_main)


I have investigated the notable financial news and potential investment opportunities.

**Key Financial News:**

*   Nvidia's upcoming earnings report is a significant event for the AI stock market.
*   Intel's stock experienced a jump due to news regarding a potential US government stake.
*   The overall market is experiencing volatility due to interest rate policies and other economic factors.

**Potential Investment Opportunities:**

*   **Momentum Stocks:** VRT, MTZ, and HOOD were identified as having strong yearly gains but recent pullbacks.
*   **Analyst Mentions:** The Mosaic Company (MOS), Hexcel Corporation (HXL), Portland General Electric Company (POR), and Dollar General Corporation (DG) were mentioned in recent analyst reports.

I attempted to look up the share prices for Intel (INTC), VRT, MTZ, and HOOD to inform my trading decisions. However, the `lookup_share_price` tool returned a price of 0.0 for all these symbols. Without accurate and current stock prices, I am unable

I have investigated the notable financial news and potential investment opportunities.

**Key Financial News:**

*   Nvidia's upcoming earnings report is a significant event for the AI stock market.
*   Intel's stock experienced a jump due to news regarding a potential US government stake.
*   The overall market is experiencing volatility due to interest rate policies and other economic factors.

**Potential Investment Opportunities:**

*   **Momentum Stocks:** VRT, MTZ, and HOOD were identified as having strong yearly gains but recent pullbacks.
*   **Analyst Mentions:** The Mosaic Company (MOS), Hexcel Corporation (HXL), Portland General Electric Company (POR), and Dollar General Corporation (DG) were mentioned in recent analyst reports.

I attempted to look up the share prices for Intel (INTC), VRT, MTZ, and HOOD to inform my trading decisions. However, the `lookup_share_price` tool returned a price of 0.0 for all these symbols. Without accurate and current stock prices, I am unable to make informed buying or selling decisions as per my day trading strategy. Therefore, I have not made any trades at this time.

In [107]:
from traders import Trader

In [108]:
trader = Trader("Himanshu",model_name="gemini-2.5-flash")

In [109]:
await trader.run()

In [110]:
await read_accounts_resource("Himanshu")

'{"name": "himanshu", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-26 23:00:43", 10000.0], ["2025-08-26 23:00:49", 10000.0], ["2025-08-26 23:03:19", 10000.0], ["2025-08-26 23:06:22", 10000.0], ["2025-08-26 23:08:26", 10000.0], ["2025-08-26 23:11:35", 10000.0], ["2025-08-26 23:15:08", 10000.0], ["2025-08-26 23:19:41", 10000.0], ["2025-08-26 23:20:59", 10000.0], ["2025-08-26 23:21:39", 10000.0], ["2025-08-26 23:39:20", 10000.0], ["2025-08-26 23:40:51", 10000.0], ["2025-08-26 23:41:43", 10000.0], ["2025-08-26 23:56:33", 10000.0], ["2025-08-26 23:57:26", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}'

<h3>Count of tools</h3>

In [112]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 5 MCP servers, and 12 tools
